In [1]:
import os
path_pipest = os.path.abspath('./')
n=0
while (not os.path.basename(path_pipest)=='pipest') and (n<6):
    path_pipest=os.path.dirname(path_pipest)
    n+=1 
if not os.path.basename(path_pipest)=='pipest':
    raise ValueError("path_pipest not found. Instead: {}".format(path_pipest))
path_sdhawkes=path_pipest+'/sdhawkes_powerlaw'
path_lobster=path_pipest+'/lobster_for_sdhawkes'
path_lobster_data=path_lobster+'/data'
path_lobster_pyscripts=path_lobster+'/py_scripts'
path_models=path_pipest+'/models'
path_tests=path_pipest+'/tests'
path_saved_tests=path_tests+'/saved_tests'


import time
import datetime
import sys
sys.path.append(path_sdhawkes+'/resources/')
sys.path.append(path_sdhawkes+'/modelling/')
sys.path.append(path_lobster_pyscripts+'/')


import pickle
import numpy as np
import pandas as pd
import bisect
import copy


import model
import lob_model
import minimisation_algo as minim_algo
import mle_estimation as mle_estim


In [2]:

list_of_n_states=[3,5]
n_events = 4 
n_levels = 2

sd_model = model.SDHawkes(number_of_event_types=n_events,
                 number_of_lob_levels=n_levels,
                 volume_imbalance_upto_level=2,
                 list_of_n_states=list_of_n_states
                )

tot_n_states=sd_model.state_enc.tot_n_states

# The base rates $\nu$
nus = 0.002*np.random.randint(low=2,high=6,size=n_events)
# The impact coefficients $\alpha$
alphas = np.random.uniform(0.0002,0.2435,size=(n_events, tot_n_states, n_events)).astype(np.float)
# The decay coefficients $\beta$
betas = np.random.uniform(1.265,1.805,size=(n_events, tot_n_states, n_events)).astype(np.float)
sd_model.set_hawkes_parameters(nus,alphas,betas)
# The transition probabilities $\phi$
phis = sd_model.state_enc.generate_random_transition_prob(n_events=n_events).astype(np.float)
sd_model.set_transition_probabilities(phis)
#     sd_model.enforce_symmetry_in_transition_probabilities()
# The Dirichlet parameters $\kappa$
kappas = np.random.lognormal(size=(tot_n_states,2*n_levels))
sd_model.set_dirichlet_parameters(kappas)


time_start = 0.0
time_end = time_start + 2*60*60
max_number_of_events = 4000



print("\nSIMULATION\n")

times, events, states, volumes = sd_model.simulate(
    time_start, time_end,max_number_of_events=max_number_of_events,
    add_initial_cond=True,
    store_results=True, report_full_volumes=False)
time_end=float(times[-1])

sd_model.create_goodness_of_fit(type_of_input='simulated')
sd_model.goodness_of_fit.ks_test_on_residuals()
sd_model.goodness_of_fit.ad_test_on_residuals()


print("\nMINIMISATION PROCEDURE\n")
event_type=0
num_init_guesses = 7
list_init_guesses = []
for n in range(num_init_guesses):
    list_init_guesses.append(
        1.0 + np.random.lognormal(size=(1+2*sd_model.number_of_event_types*sd_model.number_of_states,))
    )
MinimProc = minim_algo.MinimisationProcedure(
    sd_model.labelled_times, sd_model.count, time_start, time_end,
    sd_model.number_of_event_types,sd_model.number_of_states,
    event_type,
    list_init_guesses = list_init_guesses,
    learning_rate = 0.0004,
    maxiter = 5,
    tol = 1.0e-7,
    copy_lt = True,
)
run_time = -time.time()
MinimProc.launch_minimisation(parallel=True, return_results = False)
run_time+=time.time()
print("Minimisation terminates. run_time = {}".format(run_time))


print("\nESTIMATION HAWKES PARAMETERS\n")
run_time = -time.time()
base_rate,imp_coef,dec_coef=mle_estim.estimate_hawkes_param_partial(
    event_type,
    sd_model.number_of_event_types, sd_model.number_of_states,
    sd_model.sampled_times,
    sd_model.sampled_events,
    sd_model.sampled_states,
    time_start,
    time_end,
    sd_model.labelled_times,
    sd_model.count,
    list_init_guesses = list_init_guesses,
    num_additional_guesses = 4,
    maxiter = 5,
    learning_rate = 0.0005,
    pre_estim_ord_hawkes = True,
    return_minim_proc = 0,
    parallel= True,
    pre_estim_parallel = True,
    copy_lt = False,
    prepare_list_of_guesses = False,
    print_list = False,
    timeout= 20.0
)
run_time+=time.time()
print("Estimation of hawkes parameters terminates. run_time={}".format(run_time))
#     print("base_rate={}".format(base_rate))
#     print("imp_coef={}".format(imp_coef))
#     print("dec_coef={}".format(dec_coef))

Initialising an instance of SDHawkes
Hawkes parameters have been set
sd_hawkes: asymmetry in transition_probabilities = 29.278764127695737
Transition probabilities have been set
Dirichlet parameters have been set

SIMULATION

Simulation is being performed on the following machine:
 posix.uname_result(sysname='Linux', nodename='claudio-HP-EliteBook-850-G3', release='5.3.0-42-generic', version='#34~18.04.1-Ubuntu SMP Fri Feb 28 13:42:26 UTC 2020', machine='x86_64')
sd_hawkes: simulate: initial conditions have been acknowledged
I am adding initial conditions on the negative time axis
sd_hawkes_powerlaw_simulation.simulate: start of initialisation
   Number of levels in the order book: 2
   number_of_initial_events=61
sd_hawkes_powerlaw_simulation: simulate: labelled_times and count have been initialised.
sd_hawkes_powerlaw_simulation: simulate: intensities have been initialised.
  intensities at start: [0.01  0.008 0.008 0.008]
  intensity_overall at start: 0.034
sd_hawkes_powerlaw_simula

In [3]:
MinimProc.results

[{'x_min': array([1.00000000e-07, 1.92658171e+00, 1.86538898e+00, 1.40811256e+00,
         1.96145577e+00, 1.92413086e+00, 1.38185819e+00, 1.62850936e+00,
         1.39199936e+00, 1.29223663e+00, 1.99349470e+00, 1.28681355e+00,
         1.59417300e+00, 1.78792778e+00, 1.92988680e+00, 1.87837743e+00,
         1.91205614e+00, 1.31301779e+00, 1.99733932e+00, 1.97156108e+00,
         1.93184509e+00, 1.96238264e+00, 1.41460586e+00, 1.97141416e+00,
         1.95557304e+00, 1.99254290e+00, 1.95361576e+00, 1.92892052e+00,
         1.98868028e+00, 1.97788811e+00, 1.97562132e+00, 1.20247607e+00,
         1.96061276e+00, 1.84388023e+00, 1.94401947e+00, 1.46012482e+00,
         1.56204843e+00, 1.69546017e+00, 1.65038803e+00, 1.70869624e+00,
         1.99740618e+00, 1.16512808e+00, 1.91172391e+00, 1.85055842e+00,
         1.05396800e+00, 1.89271155e+00, 1.40166400e+00, 1.92538806e+00,
         1.97542435e+00, 1.48328907e+00, 1.73692713e+00, 1.94217344e+00,
         1.96841708e+00, 1.69442410e+00, 1

In [4]:
base_rate

array([1.e-07])